In [155]:
import requests
import pandas as pd
import numpy as np
import pulp

# Use API to Grab Current Team

In [156]:
history = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df.pkl')
current_gw = int(history[history['finished'] == True].sort_values('kickoff_time')['gw'].tail(1))

In [157]:
url = f'https://fantasy.premierleague.com/api/entry/5171826/event/{current_gw}/picks/'
r = requests.get(url)
json = r.json()

In [158]:
current_team = pd.DataFrame(json['picks'])
money_remaining = json['entry_history']['bank']

In [159]:
current_team = current_team.merge(history[['element', 'player']], how='left').drop_duplicates().reset_index(drop=True)

# Solve Knapsack Problem

In [160]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/processed/grouped_predictions_with_cost.pkl')

In [183]:
#currently not calculating for change in player value

df.gw = df.gw.astype('int64')
data = df[df.gw.isin(df.gw.unique()[:5])] #filter for next 5 gws only
data = data.groupby(['player', 'position', 'team', 'now_cost', 'element'])['predicted_points'].sum().reset_index()

expected_scores = data.predicted_points.values
num_players = len(data)
buy_prices = data.now_cost.values / 10
sell_prices = data.now_cost.values / 10
positions = data.position.values
teams = data.team.values
names = data.player.values
budget_now = money_remaining/10

In [184]:
from transfer_optimizer import TransferOptimiser

In [185]:
opt = TransferOptimiser(expected_scores, buy_prices, sell_prices, positions, teams)

In [186]:
current_squad_indices = data[data.element.isin(current_team.element)].index #look up indices for current team

In [187]:
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(current_squad_indices, budget_now, sub_factor=0.2)

Solver status: 1


In [188]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {:.2f}".format(names[i], buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {:.2f}".format(names[i], sell_prices[i], expected_scores[i]))

Transferred in: Anthony_Knockaert 5.0 34.15
Transferred out: Ben_Mee 5.1 19.47
Transferred out: Bruno Miguel_Borges Fernandes 11.6 29.65
Transferred in: Craig_Dawson 4.5 45.17
Transferred in: Luke_Thomas 4.2 37.09
Transferred out: Mohamed_Salah 12.6 13.14
Transferred in: Richarlison_de Andrade 7.9 32.46
Transferred in: Rob_Holding 4.5 38.12
Transferred out: Rúben Santos_Gato Alves Dias 6.1 26.80
Transferred in: Tomas_Soucek 5.3 57.97
Transferred in: Wilfred_Ndidi 4.8 36.23


In [189]:
player_indices = []
print()
print("First Team:")
for i in range(len(starters)):
    if starters[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


First Team:
Craig_Dawson
Dominic_Calvert-Lewin
Emiliano_Martínez
Jack_Grealish
Luke_Thomas
Ollie_Watkins
Pedro_Lomba Neto
Rob_Holding
Tomas_Soucek*
Vladimir_Coufal
Wilfred_Ndidi

Subs:
Alex_McCarthy
Anthony_Knockaert
João Pedro Cavaco_Cancelo
Richarlison_de Andrade
